In [277]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor,MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from scipy.spatial import cKDTree
from sklearn.svm import SVR
import geopandas as gpd
import operator 
import matplotlib.pyplot as plt
from sklearn import datasets
import matplotlib.pyplot as plt
import math
from pyogrio import read_dataframe, write_dataframe
from simpledbf import Dbf5
import math

# This step aims to remove repeated parcels in the dataset and save it to Parcels_tf6_5_area_unique


In [247]:

fp = "../inputs/data_export/Parcels_tf6_5_area.shp"
Geodata = read_dataframe(fp, index_col=0)


/Users/yuhan/opt/anaconda3/envs/hazard_env/lib/python3.11/site-packages/pyogrio/raw.py:137: RuntimeWarning: driver ESRI Shapefile does not support open option INDEX_COL
  result = ogr_read(


In [264]:
a = 134775
b = 165146
print( Geodata.loc[a].NAME )
print( Geodata.loc[b].NAME )

print( Geodata.loc[a].NAME == Geodata.loc[b].NAME )


MATTERN JOHN L & BEVERLY A
GU QING
False


In [262]:
Geodata.loc[151272].NAME

'WALKER BEATRICE'

In [248]:
print(Geodata.columns)
Geodata["FID"] = Geodata.index

Index(['OBJECTID', 'GEOID', 'ID', 'NAME', 'CITY', 'ST', 'ZIP', 'SITUS_NO',
       'SITUS', 'LEGAL',
       ...
       'geoid_1', 'lng', 'lat', 'Accessb', 'logAccess', 'release',
       'capture_da', 'area_2', 'pararea', 'geometry'],
      dtype='object', length=125)


In [253]:
#dbf_poly_neighbors = Dbf5('../inputs/data_export/parcel_poly_neighbors.dbf')

dbf_poly_neighbors = Dbf5('../inputs/data_export/parcel_poly_neighbors_updated.dbf')
df_poly_neighbors  = dbf_poly_neighbors.to_dataframe()

In [254]:
print( df_poly_neighbors.head() )

   src_index  nbr_index  AREA      LENGTH  NODE_COUNT
0          0      82181   0.0   92.260318           0
1          0      82231   0.0  182.571799           0
2          0      82248   0.0   88.505310           0
3          1      82224   0.0   24.510566           0
4          1      82231   0.0   43.855657           0


In [255]:
print( df_poly_neighbors.shape )
df_neighbors_select = df_poly_neighbors.loc[ df_poly_neighbors['AREA'] > 0  ]
print( df_neighbors_select.shape )


(636780, 5)
(36, 5)


In [44]:
src_FID_list = df_neighbors_select.src_FID.unique()
neighorlist = []
for src_fid in src_FID_list:
    x = df_poly_neighbors.loc[ ( df_poly_neighbors['src_FID'] == src_fid) &
                               ( df_poly_neighbors['pararea'] > 0 ) , 'nbr_FID' ].tolist()
    if len(x) > 0:
        #print(x)
        x.append( src_fid )
        neighorlist.append( set(x) )
        #print(x)
        #break


In [45]:
print( len(neighorlist) )

131654


In [61]:
repeat_neighorlist = list( set( frozenset(item) for item in neighorlist ))

In [62]:
print( len( repeat_neighorlist ) )


55377


In [63]:
#repeat_neighorlist

In [64]:
print( list( repeat_neighorlist[2] ) )

print( )
    
np.unique( Geodata.loc[ Geodata['FID'].isin( 
    list( repeat_neighorlist[2] ) ) ].pararea.values )


[144858, 144859]



array([4597.07])

In [65]:
Geodata.columns

Index(['OBJECTID', 'GEOID', 'ID', 'NAME', 'CITY', 'ST', 'ZIP', 'SITUS_NO',
       'SITUS', 'LEGAL',
       ...
       'lng', 'lat', 'Accessb', 'logAccess', 'release', 'capture_da', 'area_2',
       'pararea', 'geometry', 'FID'],
      dtype='object', length=126)

In [212]:
nonrepreat_list = []
unique_rows     = []
iter1 = 0
iter2 = 0
print( "Total repreated set is {}".format( len( repeat_neighorlist ) ) )
for row in repeat_neighorlist:
    if iter1 % 20000 == 0:
        print( iter1 )
    neighborsi = list( row )
    filtered = Geodata.loc[ Geodata['FID'].isin( neighborsi ) ]
    neighbor_areas = filtered.pararea.values
    neighbor_names = filtered.NAME.values

    ratio_arg = np.min(neighbor_areas)*1.0 / np.max(neighbor_areas)
    if len( np.unique( neighbor_areas ) ) == 1 :
        result_df = filtered.drop_duplicates( 
                subset=['pararea'] ).to_dict('records')
        unique_rows.append( result_df[0] )
        nonrepreat_list.extend( neighborsi )
    elif len( np.unique( neighbor_names ) ) == 1:
        result_df = filtered.drop_duplicates( 
                subset=['NAME'] ).to_dict('records')
        unique_rows.append( result_df[0] )
        nonrepreat_list.extend( neighborsi )
    elif ratio_arg > 0.9 :
        #index_min = np.argmin( neighbor_areas )
        index_max = np.argmax( neighborsi )
        #kept_neighbor = neighborsi[index_min]
        kept_neighbor = neighborsi[ index_max ]
        result_df2 = Geodata.loc[ Geodata['FID'] == kept_neighbor ].to_dict('records')
        #print( Geodata.loc[ Geodata['FID'].isin( neighborsi ) ] )
        unique_rows.append( result_df2[0] )
        # print( result_df2 )
        # print( Geodata.loc[ Geodata['FID'] == kept_neighbor][["LANDUSE", "EXEMPT",
        #                     "VAL19TOT", "VAL19LAND",
        #                     "Bldg_Value", "Prop_Use",
        #                     "Occupancy", "H_ft"]]  )
        #neighborsi_update = neighborsi.remove( kept_neighbor )
        nonrepreat_list.extend( neighborsi )
        #break
    elif ratio_arg > 0 and ratio_arg <= 0.9:
        index_max = np.argmax( neighbor_areas )
        #index_max = np.argmax( neighborsi )
        kept_neighbor = neighborsi[index_max]
        result_df2 = Geodata.loc[ Geodata['FID'] == kept_neighbor ].to_dict('records')
        #print( Geodata.loc[ Geodata['FID'].isin( neighborsi ) ] )
        unique_rows.append( result_df2[0] )
        nonrepreat_list.extend( neighborsi )    
    else:
        iter2 = iter2 +1
        # print( Geodata.loc[ Geodata['FID'] == kept_neighbor][["LANDUSE", "EXEMPT",
        #                     "VAL19TOT", "VAL19LAND",
        #                     "Bldg_Value", "Prop_Use",
        #                     "Occupancy", "H_ft"]]  )
        #neighborsi_update = neighborsi.remove( kept_neighbor )
    #nonrepreat_list.extend( neighborsi )
    iter1       = iter1 + 1
unique_rows_df  = pd.DataFrame( unique_rows )
kept_rows       = Geodata.drop( Geodata[Geodata['FID'].isin( nonrepreat_list ) ].index )


Total repreated set is 55377
0
20000
40000


In [213]:
iter2

0

In [221]:
neighbor_area = Geodata.loc[ Geodata['FID'].isin( neighborsi ) ].pararea.values
neighbor_area

array([6665.5, 6665.5])

In [222]:
frames = [ kept_rows, unique_rows_df]
result = pd.concat(frames).reset_index()
print(result.shape)
print("Updated results is {} ".format(result.shape[0]) )
print("Geodata shape is {} ".format(Geodata.shape[0]) )


(170194, 127)
Updated results is 170194 
Geodata shape is 246471 


In [223]:
result['index'] = result.index
result_gdf = gpd.GeoDataFrame(result, crs="EPSG:4326", geometry = 'geometry')


In [224]:
result_gdf.tail()

,index,OBJECTID,GEOID,ID,NAME,CITY,ST,ZIP,SITUS_NO,SITUS,...,lng,lat,Accessb,logAccess,release,capture_da,area_2,pararea,geometry,FID
170189,170189,196276,3510-0046-3012-000,106526,VANDERHOOFVEN JON WESLEY & BETH ANN,MISSOURI CITY,TX,77459-7649,1520,"1520 AVE O GALVESTON, TX 77550",...,-94.784394,29.299008,11646.0,9.362718,2,11/16/2019-11/18/2019,53.410,480.785,"POLYGON ((-94.78137 29.29622, -94.78150 29.296...",95294
170190,170190,295046,2492-0002-0015-000,395053,GUSS STEPHEN J & EVELYN,LEAGUE CITY,TX,77573,409,"409 PRATTWOOD CT LEAGUE CITY, TX 77573",...,-95.150475,29.479846,118697.0,11.684329,2,11/19/2019-11/24/2019,264.898,677.994,"POLYGON ((-95.13451 29.49207, -95.13450 29.491...",130450
170191,170191,216175,2775-0011-3103-001,196021,HOGAN FRANK G,LA MARQUE,TX,77568-6228,409,"409 CHERRY LA MARQUE, TX 77568",...,-94.914202,29.358818,12630.0,9.443830,2,11/19/2019-11/24/2019,284.289,740.597,"POLYGON ((-94.95237 29.36354, -94.95256 29.363...",81626
170192,170192,297933,2815-0009-0029-000,145223,HUNTER ERIC M,LEAGUE CITY,TX,77573-1684,6220,"6220 CREEKSIDE LN LEAGUE CITY, TX 77573",...,-95.140503,29.499595,22690.0,10.029680,2,11/19/2019-11/24/2019,205.643,672.292,"POLYGON ((-95.15973 29.49534, -95.16000 29.495...",121691
170193,170193,240342,7042-0000-0047-002,188710,ISTAR BOWLING CENTERS II LP,NEW YORK,NY,10036,2404,"2404 PALMER HWY TEXAS CITY, TX 77590",...,-94.931012,29.401320,26282.0,10.176640,2,11/19/2019-11/24/2019,2649.362,6665.500,"POLYGON ((-94.92911 29.39482, -94.92911 29.394...",113271


In [225]:
result_gdf['CensusBlk']     = result_gdf["CensusBlk"].astype(int).apply(str)
result_gdf['CensusBloc']    = result_gdf["CensusBlk"].astype(int).apply(str)

result_gdf['BUILDVALUE']    = result_gdf["BUILDVALUE"].astype(int)
result_gdf['CFLD_EXPB']     = result_gdf["CFLD_EXPB"].astype(int)
result_gdf['CFLD_EXPT']     = result_gdf["CFLD_EXPT"].astype(int)

result_gdf['VAL19IMP']      = result_gdf["VAL19IMP"].astype(int)
result_gdf['VAL19TOT']      = result_gdf["VAL19TOT"].astype(int)
result_gdf['Bldg_Value']    = result_gdf["Bldg_Value"].astype(int)

In [226]:
print( result_gdf.VAL19IMP.head() )


0     27730
1    300000
2         0
3    214170
4     36020
Name: VAL19IMP, dtype: int64


In [85]:
#result_gdf.to_file(filename= "../inputs/data_export/Parcels_tf6_5_area_unique0.shp", 
#                   driver='ESRI Shapefile')

In [227]:
write_dataframe(result_gdf, 
                path = "../inputs/data_export/Parcels_tf6_5_area_unique.shp", 
                driver='ESRI Shapefile')

# This step will read the unique parcels and their neighbors

In [228]:
fp = "../inputs/data_export/Parcels_tf6_5_area_unique.shp"
Geodata2 = read_dataframe(fp, index_col=0)
Geodata2["TARGET_FID"] = Geodata2["index"]


/Users/yuhan/opt/anaconda3/envs/hazard_env/lib/python3.11/site-packages/pyogrio/raw.py:137: RuntimeWarning: driver ESRI Shapefile does not support open option INDEX_COL
  result = ogr_read(


In [229]:
dbf_neighbors_updated = Dbf5('../inputs/data_export/parcel_poly_neighbors_updated.dbf')
df_neighbors_updated  = dbf_neighbors_updated.to_dataframe()


In [235]:
df_neighbors_updated.head()

,src_index,nbr_index,AREA,LENGTH,NODE_COUNT
0,0,82181,0.0,92.260318,0
1,0,82231,0.0,182.571799,0
2,0,82248,0.0,88.505310,0
3,1,82224,0.0,24.510566,0
4,1,82231,0.0,43.855657,0


In [241]:
Geodata2.NEIGHBORS
#Geodata2.head()

0          
1          
2          
3          
4          
         ..
170189     
170190     
170191     
170192     
170193     
Name: NEIGHBORS, Length: 170194, dtype: object

In [288]:
Geodata3 = Geodata2.copy()
updated_neibor = []
countx = 0
for idi, rowi in Geodata3.iterrows():
    rowi_neighbors = df_neighbors_updated.loc[ 
        df_neighbors_updated['src_index'] == rowi['TARGET_FID'] ].nbr_index.values
    #print( rowi_neighbors )
    if pd.isnull( rowi.LANDUSE ):
        countx = countx + 1
        neighbor_str = ''.join( str(e) for e in rowi_neighbors )
        updated_neibor.append( neighbor_str ) 
    else:
        row_landuse = set( rowi.LANDUSE.split(',') )
        rowi_updated_neighbors = []
        for e in rowi_neighbors:
            rowi_land_feature = Geodata3.loc[ Geodata3['TARGET_FID'] == e ].LANDUSE.values[0]
            if not pd.isnull( rowi_land_feature ):
                landuse_cates = set( rowi_land_feature.split(',') )
                if len( row_landuse.intersection( landuse_cates ) ) > 0:
                    rowi_updated_neighbors.append(e)
        neighbor_str = ','.join( str(e) for e in rowi_updated_neighbors )
        updated_neibor.append( neighbor_str ) 
    if idi % 10000 == 0:
        print("Index is {}".format( idi ) )
Geodata3[ 'NEIGHBORS' ] = updated_neibor

Index is 0
Index is 10000
Index is 20000
Index is 30000
Index is 40000
Index is 50000
Index is 60000
Index is 70000
Index is 80000
Index is 90000
Index is 100000
Index is 110000
Index is 120000
Index is 130000
Index is 140000
Index is 150000
Index is 160000
Index is 170000


In [289]:
countx


3139

In [290]:
Geodata3.NEIGHBORS

0                                     82231
1                                          
2                              86011,130210
3                                    140068
4                                         5
                        ...                
170189                         38750,153661
170190           49075,126617,126809,165393
170191                  89278,154866,160735
170192    45249,134369,134855,141671,142777
170193                                69412
Name: NEIGHBORS, Length: 170194, dtype: object

In [291]:
write_dataframe(Geodata3, 
                path = "../inputs/data_export/Parcels_tf6_5_area_unique.shp", 
                driver='ESRI Shapefile')

/Users/yuhan/opt/anaconda3/envs/hazard_env/lib/python3.11/site-packages/pyogrio/raw.py:456: RuntimeWarning: Value '965512930129331293512936129381293912943129481295012951129581058221058261058271058291058301058331058381058461058491058511058521058531058541058561058571058601058611058621058631058651058671058681058691058701058711058731058751058771058781058791058801058811058821058831058841058851058861058871058881058891058901058911058931058941058951058961058991059011059041059091059121063191063451063481063661063671063741063821063891064121064131064271064281064301064331064371064441064541064591064661064681064751064781064831064901064981065001065111065171065211065281065431065481065531065571065601065641065711065761065811065841065901065911065921065941065971066051066061066071066091066161066181066231066241066251066271066311066321066331066391066401066441066531066551066591066601066611066641066661066681066691066711066791066931067031067071067111067151067161067181067231067261067291067311067331067381067391067

In [292]:
','.join( str(e) for e in [2,3,4,5,5] )


'2,3,4,5,5'

In [294]:
Geodata3.loc[ Geodata3['NEIGHBORS'] == "" ].LANDUSE.unique()

array(['CL', 'PF, SF', 'PU', 'CR', 'RL', 'RH', 'BF', 'RF', 'UN, UW, WF',
       'DE', 'CO', 'WF', 'R1, RL', 'BV', 'BA, RF', 'UW', 'GB', 'RF, RH',
       'RA', 'RF, RW', 'F5, RH', 'PF', 'RS, WF', 'PF, RS', 'F2', 'E1',
       'D1', 'UN', 'C1', 'RW', 'BA, PF, PL, RS', 'RF, RL', 'PF, PL', 'F4',
       'BF, PU', 'UW, WF', 'CL, D1', 'DE, RL', 'RA, RL', 'WV', 'CO, D1',
       'DE, RF', 'PWL, RL', 'PL, RL', 'D1, RH', 'IL, RL', 'PF, RL',
       'E1, RF', 'E1, RH', 'RH, UN', 'GF', 'GC', 'F8, RL', 'RH, RS',
       'DE, RH', 'BF, BV', 'ML, WF', 'RH, RL', 'CL, PF', 'RL, WV',
       'E2, RL', 'LT', 'RA, WL', 'RL, WF', 'PF, UN, UW',
       'CN, DE, PF, RS, SF', 'BA, PF', 'PL, RF', 'F2, RA', 'C1, RH',
       'DE, PF, SF', 'PF, RS, SF', 'BA, RF, RH', 'CL, WF',
       'D1, E1, RH, UW', 'WF, WV', 'CN', 'RS', 'BA', 'RS, WV', 'TL',
       'ML, RH', 'CN, RL', 'OT', 'RH, RW', 'CL, CR', 'RF, UW', 'D1, PF',
       'RA, RH', 'RF, WF', 'PF, UN', 'CO, UN', 'DE, RF, RS', 'F8',
       'RL, UW', 'PU, UN', 'D1, RF, S